# Processing Quake data
0. combine all space data together (moon, mars, train and test data)
    - just use marks to distinguish them
    - > [Mars InSight Seismic Data](https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/data/xb/) This NASA PDS repository, hosted by the PDS Geosciences Node, contains continuous waveform data for the mission.
      > 
      > [Mars InSight Seismic Data Information Sheet](https://pds-geosciences.wustl.edu/insight/urn-nasa-pds-insight_seis/readme.txt): Contains data information for Mars InSight Seismic Instrument (hosted by the PDS Geosciences Node).
      - is there new data or we already have them?
1. mix up other data
    - [Explore data from ESA's Solar System missions](https://psa.esa.int/psa/#/pages/home) -- Planetary Science Archive: The European Space Agency's Planetary Science Archive (PSA) is the central repository for all scientific and engineering data returned by ESA's Solar System missions: currently ExoMars 2016, Giotto, Huygens, Mars Express, Rosetta, SMART-1, BepiColombo and Venus Express, as well as several ground-based cometary observations.
      - is there anything useful?
2. mix up some earth data
    - [Earthquakes Canada Insights by Natural Resources Canada](https://www.earthquakescanada.nrcan.gc.ca/index-en.php): Natural Resources Canada page on earthquakes in Canada including the data, hazards, and general information. This could potentially be useful to help you understand how earthquake data works.
    - [Earth Seismogram Viewer](https://www.earthquakescanada.nrcan.gc.ca/stndon/wf-fo/index-en.php): This site shows you how earthquake data is reported on in Canada. This could be helpful in understanding earthquake data generally.
4. visualize
    - waveform
    - spectral [link](https://docs.obspy.org/tutorial/code_snippets/continuous_wavelet_transform.html)
    - play sound of waveform
5. extract all events time series
6. extract all non-events time series
7. create dataset of events

## Imports

In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
import sys
import os
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).joinpath('..', 'src').resolve()))

In [203]:
import hashlib
import numpy as np
import obspy
import os
import pandas as pd
import pickle
from tqdm.autonotebook import tqdm

seismic.__version__

/tmp/ipykernel_51783/1604621303.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


'0.1.4'

## Setup

### Preporation steps

Run to find location of dataset
```
# Run to find location of dataset
!ls -lah /var/datasets/3/space_apps_2024_seismic_detection/

# Create directory for input data
!mkdir -p /var/datasets/3/space_apps_2024_seismic_detection/raw

# Create directory for temporary data
!mkdir -p /var/datasets/3/space_apps_2024_seismic_detection/interim

# Create directory for resulting data
!mkdir -p /var/datasets/3/space_apps_2024_seismic_detection/processed

# Link Temporary data
!ln -s /var/datasets/3/space_apps_2024_seismic_detection/raw ../data/raw/remote

# Link Temporary data
!ln -s /var/datasets/3/space_apps_2024_seismic_detection/interim ../data/interim/remote

# Link Resulting data
!ln -s /var/datasets/3/space_apps_2024_seismic_detection/processed ../data/processed/remote

# Download dataset
!wget https://wufs.wustl.edu/SpaceApps/data/space_apps_2024_seismic_detection.zip --output-document=/var/datasets/3/space_apps_2024_seismic_detection/space_apps_2024_seismic_detection.zip

# Unpack dataset
!unzip /var/datasets/3/space_apps_2024_seismic_detection/space_apps_2024_seismic_detection.zip -d /var/datasets/3/space_apps_2024_seismic_detection/raw/
```

There is mistyping in training data and we need to rename one file

existing file is:
`xa.s12.00.mhz.1971-04-13HR02_evid00029.mseed`

and correct one is:
`xa.s12.00.mhz.1971-04-13HR00_evid00029.mseed`

```bash
mv ../data/raw/remote/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR02_evid00029.mseed ../data/raw/remote/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR00_evid00029.mseed

mv ../data/raw/remote/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR02_evid00029.csv ../data/raw/remote/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR00_evid00029.csv
```

```
!cat /var/datasets/3/space_apps_2024_seismic_detection/raw/space_apps_2024_seismic_detection/welcome.txt
```

```
Welcome, Space Apps challengers, to �Seismic Detection across the Solar System�. 

Today, we challenge YOU to parse through seismic data collected on the Moon and Mars and figure out how to detect moonquakes and marsquakes! 
To get you started on the data, we present to you a training set containing the following:

1. A catalog of quakes identified in the data
2. Seismic data collected by the Apollo (one day segments) or InSight (one hour segments) missions in miniseed and CSV format. 
3. Plots of the trace and spectrogram for each day a quake has been identified. 

Using these tools, your task is to come up with an algorithm to identify these signals (using the catalog as a benchmark) and then 
apply your algorithm to the seismic data in the "test" folder (Apollo 12, 15, and 16 for the Moon and other InSight events for Mars). 

Each trace included in the test folder has a quake in it, there are no empty traces. 

This main folder also has a Jupyter notebook that will help you get started on the data.

** IMPORTANT **
Please make sure that your output catalog has at least the following headers:
filename
time_abs(%Y-%m-%dT%H:%M:%S.%f) or time_rel(sec)

If not, your output catalog may not be scored!!
** IMPORTANT **

Good luck!!! If you have any questions, our team leaders will be around to help during the hackathon weekend!
```

## Parameters

In [2]:
input_path = '../data/raw/remote'
output_path = '../data/interim/remote'

## Combine Moon and Mars Data

### Moon

#### Catalog

In [3]:
moon_train_path = f'{input_path}/space_apps_2024_seismic_detection/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
moon_catalog_df = pd.read_csv(moon_train_path)
moon_catalog_df['body'] = 'Moon'
moon_catalog_df['subset'] = 'train'
moon_catalog_df

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq,Moon,train
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq,Moon,train
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq,Moon,train
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq,Moon,train
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq,Moon,train
...,...,...,...,...,...,...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156,1974-10-14T17:43:00.000000,63780.0,evid00156,impact_mq,Moon,train
72,xa.s12.00.mhz.1975-04-12HR00_evid00191,1975-04-12T18:15:00.000000,65700.0,evid00191,impact_mq,Moon,train
73,xa.s12.00.mhz.1975-05-04HR00_evid00192,1975-05-04T10:05:00.000000,36300.0,evid00192,impact_mq,Moon,train
74,xa.s12.00.mhz.1975-06-24HR00_evid00196,1975-06-24T16:03:00.000000,57780.0,evid00196,impact_mq,Moon,train


#### Seed files

In [4]:
seed_path = f'{input_path}/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA'
extension = '.mseed'
moon_catalog_df['mseed_path'] = moon_catalog_df['filename'].apply(lambda x: f'{seed_path}/{x}{extension}')

In [5]:
def extract_mseed_stats(row):
    st = obspy.read(row['mseed_path'])
    extract_fields = [
        # 
        # example 6.625
        'sampling_rate', 
        # REMOVED (Duplicate)
        # This is the time interval between successive samples, calculated as the inverse of the sampling rate (1 / 6.625).
        # example 0.1509433962264151
        # 'delta',
        # 
        # example 1970-01-19T00:00:00.665000Z
        'starttime', 
        #
        # example 1970-01-20T00:00:02.778208Z
        'endtime', 
        # REMOVED (Duplicate)
        # The total number of data points (samples) in the dataset. This helps in determining the total length of your data when combined with delta.
        # example 572415
        # 'npts', 
        # This is a calibration factor, which tells you how to convert the raw data into physical units like ground displacement, velocity, or acceleration. Here, it’s set to 1.0, meaning the data is already in the correct units.
        # example 1.0
        'calib', 
        #
        # example 'XA'
        'network', 
        #
        # example 'S12'
        'station', 
        #
        # example '00'
        'location', 
        # The channel code defines what type of seismic data this is. Here, MHZ might refer to a high-frequency vertical component of ground motion.
        # example 'MHZ'
        'channel', 
    ]
    return pd.Series({f'mseed_{k}': v for k, v in st[0].stats.__dict__.items() if k in extract_fields})

moon_catalog_df = pd.concat([
    moon_catalog_df,
    moon_catalog_df.apply(extract_mseed_stats, axis=1)
], axis=1)

moon_catalog_df

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-01-19T00:00:00.665000Z,1970-01-20T00:00:02.778208Z,1.0,XA,S12,00,MHZ
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-03-25T00:00:00.440000Z,1970-03-26T00:00:01.949434Z,1.0,XA,S12,00,MHZ
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-03-26T00:00:00.565000Z,1970-03-27T00:00:02.074434Z,1.0,XA,S12,00,MHZ
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-04-25T00:00:00.196000Z,1970-04-26T00:00:02.309208Z,1.0,XA,S12,00,MHZ
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-04-26T00:00:00.660000Z,1970-04-27T00:00:02.169434Z,1.0,XA,S12,00,MHZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156,1974-10-14T17:43:00.000000,63780.0,evid00156,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1974-10-14T00:00:00.996000Z,1974-10-15T00:00:03.562038Z,1.0,XA,S12,00,MHZ
72,xa.s12.00.mhz.1975-04-12HR00_evid00191,1975-04-12T18:15:00.000000,65700.0,evid00191,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1975-04-12T00:00:00.187000Z,1975-04-13T00:00:03.507755Z,1.0,XA,S12,00,MHZ
73,xa.s12.00.mhz.1975-05-04HR00_evid00192,1975-05-04T10:05:00.000000,36300.0,evid00192,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1975-05-04T00:00:00.457000Z,1975-05-05T00:00:03.023038Z,1.0,XA,S12,00,MHZ
74,xa.s12.00.mhz.1975-06-24HR00_evid00196,1975-06-24T16:03:00.000000,57780.0,evid00196,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1975-06-24T00:00:00.239000Z,1975-06-25T00:00:01.748434Z,1.0,XA,S12,00,MHZ


### Mars

#### Catalog

In [6]:
mars_path = f'{input_path}/space_apps_2024_seismic_detection/data/mars'

catalog_1_df = pd.read_csv(mars_path + '/training/catalogs/Mars_InSight_training_catalog.csv')
display(catalog_1_df)
catalog_2_df = pd.read_csv(mars_path + '/training/catalogs/Mars_InSight_training_catalog_final.csv')
display(catalog_2_df)

mars_catalog_df = pd.merge(left=catalog_1_df, right=catalog_2_df, on='evid')
mars_catalog_df['filename'] = mars_catalog_df['filename_y'].str.replace('.csv', '', regex=False)
mars_catalog_df.drop(['time', 'filename_x', 'filename_y'], axis=1, inplace=True)
mars_catalog_df['body'] = 'Mars'
mars_catalog_df['subset'] = 'train'
display(mars_catalog_df)

,filename,time,evid
0,XB.ELYSE.02.BHV.M.2022.034.080000_evid0005.mseed,2022-02-03T08:08:27.000000,evid0005
1,XB.ELYSE.02.BHV.M.2022.002.040000_evid0006.mseed,2022-01-02T04:35:30.000000,evid0006


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid
0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.csv,2022-02-03T08:08:27.000000,507.0,evid0005
1,XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.csv,2022-01-02T04:35:30.000000,2130.0,evid0006


,evid,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),filename,body,subset
0,evid0005,2022-02-03T08:08:27.000000,507.0,XB.ELYSE.02.BHV.2022-02-03HR08_evid0005,Mars,train
1,evid0006,2022-01-02T04:35:30.000000,2130.0,XB.ELYSE.02.BHV.2022-01-02HR04_evid0006,Mars,train


#### Seed files

In [7]:
mars_train_path = f'{mars_path}/training/data'
!ls -lah $mars_train_path

total 8.6M
drwxr-xr-x 2 jovyan users 4.0K Sep  5 16:33 .
drwxr-xr-x 5 jovyan users 4.0K Sep  5 16:34 ..
-rw-r--r-- 1 jovyan users 3.8M Aug 21 10:02 XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.csv
-rw-r--r-- 1 jovyan users 572K Aug 21 10:05 XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.mseed
-rw-r--r-- 1 jovyan users 3.8M Aug 21 10:02 XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.csv
-rw-r--r-- 1 jovyan users 572K Aug 21 10:05 XB.ELYSE.02.BHV.2022-02-03HR08_evid0005.mseed


In [8]:
seed_path = f'{mars_path}/training/data'
extension = '.mseed'
mars_catalog_df['mseed_path'] = mars_catalog_df['filename'].apply(lambda x: f'{seed_path}/{x}{extension}')
mars_catalog_df = pd.concat([
    mars_catalog_df,
    mars_catalog_df.apply(extract_mseed_stats, axis=1)
], axis=1)

## Load Test data

### Moon

In [9]:
import re


In [10]:
%%time
moon_test_path = f'{input_path}/space_apps_2024_seismic_detection/data/lunar/test/data'

moon_test_data = []

walk_moon_test_data = os.walk(moon_test_path)
_, top_directories, _ = next(walk_moon_test_data)
for top_dir in sorted(top_directories):
    mseed_dir_root, _, mseed_files = next(os.walk(os.path.join(moon_test_path, top_dir)))
    for mseed_file in sorted(mseed_files):
        if mseed_file.endswith('.mseed'):
            moon_test_data.append({
                'evid': re.search(r'(evid\d+)', mseed_file).group(1),
                'filename': mseed_file.replace('.mseed', ''),
                'mseed_path': f'{mseed_dir_root}/{mseed_file}'
            })

moon_test_df = pd.DataFrame(moon_test_data)
moon_test_df = pd.concat([
    moon_test_df,
    moon_test_df.apply(extract_mseed_stats, axis=1)
], axis=1)
moon_test_df['body'] = 'Moon'
moon_test_df['subset'] = 'test'
moon_test_df

CPU times: user 472 ms, sys: 259 ms, total: 731 ms
Wall time: 730 ms


,evid,filename,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,body,subset
0,evid00006,xa.s12.00.mhz.1969-12-16HR00_evid00006,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1969-12-16T00:00:00.178000Z,1969-12-17T00:00:03.498755Z,1.0,XA,S12,00,MHZ,Moon,test
1,evid00007,xa.s12.00.mhz.1970-01-09HR00_evid00007,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-01-09T00:00:00.126000Z,1970-01-10T00:00:03.446755Z,1.0,XA,S12,00,MHZ,Moon,test
2,evid00014,xa.s12.00.mhz.1970-02-07HR00_evid00014,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-02-07T00:00:00.571000Z,1970-02-08T00:00:03.891755Z,1.0,XA,S12,00,MHZ,Moon,test
3,evid00016,xa.s12.00.mhz.1970-02-18HR00_evid00016,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-02-18T00:00:00.149000Z,1970-02-19T00:00:02.262208Z,1.0,XA,S12,00,MHZ,Moon,test
4,evid00018,xa.s12.00.mhz.1970-03-14HR00_evid00018,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-03-14T00:00:00.520000Z,1970-03-15T00:00:03.840755Z,1.0,XA,S12,00,MHZ,Moon,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,evid00249,xa.s16.00.mhz.1977-04-17HR00_evid00249,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1977-04-17T00:00:00.403000Z,1977-04-18T00:00:01.912434Z,1.0,XA,S16,00,MHZ,Moon,test
92,evid00255,xa.s16.00.mhz.1977-06-02HR00_evid00255,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1977-06-02T00:00:00.164000Z,1977-06-03T00:00:06.503623Z,1.0,XA,S16,00,MHZ,Moon,test
93,evid00443,xa.s16.00.mhz.1973-08-25HR00_evid00443,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1973-08-25T00:00:00.596000Z,1973-08-26T00:00:03.162038Z,1.0,XA,S16,00,MHZ,Moon,test
94,evid00487,xa.s16.00.mhz.1973-12-18HR00_evid00487,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1973-12-18T00:00:00.227000Z,1973-12-19T00:00:03.547755Z,1.0,XA,S16,00,MHZ,Moon,test


### Mars

In [11]:
%%time
mars_test_path = f'{input_path}/space_apps_2024_seismic_detection/data/mars/test/data'

mars_test_data = []
for mseed_file in sorted(os.listdir(mars_test_path)):
    if mseed_file.endswith('.mseed'):
        mars_test_data.append({
            'evid': re.search(r'(evid\d+)', mseed_file).group(1),
            'filename': mseed_file.replace('.mseed', ''),
            'mseed_path': os.path.join(mars_test_path, mseed_file),
        })

mars_test_df = pd.DataFrame(mars_test_data)


mars_test_df = pd.concat([
    mars_test_df,
    mars_test_df.apply(extract_mseed_stats, axis=1)
], axis=1)
mars_test_df['body'] = 'Mars'
mars_test_df['subset'] = 'test'
mars_test_df

CPU times: user 16.6 ms, sys: 0 ns, total: 16.6 ms
Wall time: 16.2 ms


,evid,filename,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,body,subset
0,evid0041,XB.ELYSE.02.BHV.2019-05-23HR02_evid0041,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2019-05-23T02:00:00.032000Z,2019-05-23T02:59:59.982000Z,1.0,XB,ELYSE,02,BHV,Mars,test
1,evid0033,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2019-07-26T12:00:00.010000Z,2019-07-26T12:59:59.960000Z,1.0,XB,ELYSE,02,BHV,Mars,test
2,evid0034,XB.ELYSE.02.BHV.2019-07-26HR12_evid0034,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2019-07-26T12:00:00.010000Z,2019-07-26T12:59:59.960000Z,1.0,XB,ELYSE,02,BHV,Mars,test
3,evid0032,XB.ELYSE.02.BHV.2019-09-21HR03_evid0032,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2019-09-21T03:00:00.037000Z,2019-09-21T03:59:59.987000Z,1.0,XB,ELYSE,02,BHV,Mars,test
4,evid0017,XB.ELYSE.02.BHV.2021-05-02HR01_evid0017,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2021-05-02T01:00:00.025000Z,2021-05-02T01:59:59.975000Z,1.0,XB,ELYSE,02,BHV,Mars,test
5,evid0011,XB.ELYSE.02.BHV.2021-10-11HR23_evid0011,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2021-10-11T23:00:00.039000Z,2021-10-11T23:59:59.989000Z,1.0,XB,ELYSE,02,BHV,Mars,test
6,evid0007,XB.ELYSE.02.BHV.2021-12-24HR22_evid0007,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2021-12-24T22:00:00.041000Z,2021-12-24T22:59:59.991000Z,1.0,XB,ELYSE,02,BHV,Mars,test
7,evid0002,XB.ELYSE.02.BHV.2022-04-09HR22_evid0002,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2022-04-09T22:00:00.035000Z,2022-04-09T22:59:59.985000Z,1.0,XB,ELYSE,02,BHV,Mars,test
8,evid0001,XB.ELYSE.02.BHV.2022-05-04HR23_evid0001,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2022-05-04T23:00:00.048000Z,2022-05-04T23:59:59.948000Z,1.0,XB,ELYSE,02,BHV,Mars,test


## TKTK: Mix up earth data
Might be useful for training

## Aggregate data

In [12]:
aggregated_df = pd.concat([
    moon_catalog_df,
    moon_test_df,
    mars_catalog_df,
    mars_test_df,
])
aggregated_df.groupby(['body', 'subset'])['mseed_path'].count().rename('Count').to_frame()

Count
body subset       
Mars test        9
     train       2
Moon test       96
     train      76

## Remove Duplication

There are few mseed files which are shared between train and test subsets:
1. One case
    - Test - `xa.s12.00.mhz.1970-07-20HR00_evid00037`
    - Train - `xa.s12.00.mhz.1970-07-20HR00_evid00011`
    - Train - `xa.s12.00.mhz.1970-07-20HR00_evid00010`
2. Other
    - Test - `xa.s12.00.mhz.1971-10-06HR00_evid00124`
    - Test - `xa.s12.00.mhz.1971-10-06HR00_evid00125`

and we have more, so instead of refering the same file we will combine them together

In [13]:
%%time
def get_hash_of_mseed(row):
    return hashlib.md5(open(row['mseed_path'],'rb').read()).hexdigest()

aggregated_df['mseed_hash'] = aggregated_df.apply(get_hash_of_mseed, axis=1)

CPU times: user 1.03 s, sys: 79.5 ms, total: 1.11 s
Wall time: 1.11 s


In [14]:
mseed_hash_duplication_df = aggregated_df['mseed_hash'].value_counts()
mseed_hash_duplication_df = mseed_hash_duplication_df[mseed_hash_duplication_df > 1]
mseed_hash_duplications = mseed_hash_duplication_df.index
mseed_hash_duplication_df

08237af0c58b3ef71046df747c82c92e    3
ac8dc1a2a5fae79a63c3eeffbe9f0f57    2
819dcdc86b9e46b594c84b4987fc24b0    2
57ec69a3a5ff33aa2fa39123dccbce55    2
aa51398b74b83aa07d43f8e0f54f177e    2
b548fa9a5c3e8a618bb37c1dd880eead    2
81453e5c02bef85b63f73a116393f948    2
7445689798d974bd70098e2f4323e2f8    2
f71fe89f14cb453f68e455de41638ab1    2
dcc1ef5de869555ea2eb81f91ebe3b1e    2
Name: mseed_hash, dtype: int64

Remap all duplications to the same mseed file

In [15]:
for duplication_hash in mseed_hash_duplications:
    duplications_df = aggregated_df[aggregated_df['mseed_hash'] == duplication_hash]
    display(duplications_df)
    aggregated_df.loc[aggregated_df['mseed_hash'] == duplication_hash, 'mseed_path'] = sorted(duplications_df['mseed_path'])[0]

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
7,xa.s12.00.mhz.1970-07-20HR00_evid00010,1970-07-20T05:06:00.000000,18360.0,evid00010,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-07-20T00:00:00.487000Z,1970-07-21T00:00:01.996434Z,1.0,XA,S12,00,MHZ,08237af0c58b3ef71046df747c82c92e
8,xa.s12.00.mhz.1970-07-20HR00_evid00011,1970-07-20T11:44:00.000000,42240.0,evid00011,deep_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-07-20T00:00:00.487000Z,1970-07-21T00:00:01.996434Z,1.0,XA,S12,00,MHZ,08237af0c58b3ef71046df747c82c92e
14,xa.s12.00.mhz.1970-07-20HR00_evid00037,NaN,NaN,evid00037,NaN,Moon,test,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-07-20T00:00:00.487000Z,1970-07-21T00:00:01.996434Z,1.0,XA,S12,00,MHZ,08237af0c58b3ef71046df747c82c92e


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
18,xa.s12.00.mhz.1970-11-03HR00_evid00050,NaN,NaN,evid00050,NaN,Moon,test,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-11-03T00:00:00.395000Z,1970-11-04T00:00:03.715755Z,1.0,XA,S12,00,MHZ,ac8dc1a2a5fae79a63c3eeffbe9f0f57
19,xa.s12.00.mhz.1970-11-03HR00_evid00051,NaN,NaN,evid00051,NaN,Moon,test,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1970-11-03T00:00:00.395000Z,1970-11-04T00:00:03.715755Z,1.0,XA,S12,00,MHZ,ac8dc1a2a5fae79a63c3eeffbe9f0f57


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
48,xa.s12.00.mhz.1973-06-05HR00_evid00107,1973-06-05T02:38:00.000000,9480.0,evid00107,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1973-06-05T00:00:00.300000Z,1973-06-05T23:59:59.394340Z,1.0,XA,S12,00,MHZ,819dcdc86b9e46b594c84b4987fc24b0
49,xa.s12.00.mhz.1973-06-05HR00_evid00108,1973-06-05T11:10:00.000000,40200.0,evid00108,deep_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1973-06-05T00:00:00.300000Z,1973-06-05T23:59:59.394340Z,1.0,XA,S12,00,MHZ,819dcdc86b9e46b594c84b4987fc24b0


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
67,xa.s12.00.mhz.1974-07-06HR00_evid00150,1974-07-06T02:57:00.000000,10620.0,evid00150,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1974-07-06T00:00:00.193000Z,1974-07-07T00:00:01.702434Z,1.0,XA,S12,00,MHZ,57ec69a3a5ff33aa2fa39123dccbce55
68,xa.s12.00.mhz.1974-07-06HR00_evid00151,1974-07-06T14:14:00.000000,51240.0,evid00151,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1974-07-06T00:00:00.193000Z,1974-07-07T00:00:01.702434Z,1.0,XA,S12,00,MHZ,57ec69a3a5ff33aa2fa39123dccbce55


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
22,xa.s12.00.mhz.1971-05-12HR00_evid00031,1971-05-12T08:05:00.000000,29100.0,evid00031,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1971-05-12T00:00:00.712000Z,1971-05-13T00:00:01.617660Z,1.0,XA,S12,00,MHZ,aa51398b74b83aa07d43f8e0f54f177e
23,xa.s12.00.mhz.1971-05-12HR00_evid00032,1971-05-12T09:45:00.000000,35100.0,evid00032,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1971-05-12T00:00:00.712000Z,1971-05-13T00:00:01.617660Z,1.0,XA,S12,00,MHZ,aa51398b74b83aa07d43f8e0f54f177e


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
1,XB.ELYSE.02.BHV.2019-07-26HR12_evid0033,NaN,NaN,evid0033,NaN,Mars,test,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2019-07-26T12:00:00.010000Z,2019-07-26T12:59:59.960000Z,1.0,XB,ELYSE,02,BHV,b548fa9a5c3e8a618bb37c1dd880eead
2,XB.ELYSE.02.BHV.2019-07-26HR12_evid0034,NaN,NaN,evid0034,NaN,Mars,test,../data/raw/remote/space_apps_2024_seismic_det...,20.0,2019-07-26T12:00:00.010000Z,2019-07-26T12:59:59.960000Z,1.0,XB,ELYSE,02,BHV,b548fa9a5c3e8a618bb37c1dd880eead


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
28,xa.s12.00.mhz.1971-10-06HR00_evid00124,NaN,NaN,evid00124,NaN,Moon,test,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1971-10-06T00:00:00.213000Z,1971-10-07T00:00:03.080925Z,1.0,XA,S12,00,MHZ,81453e5c02bef85b63f73a116393f948
29,xa.s12.00.mhz.1971-10-06HR00_evid00125,NaN,NaN,evid00125,NaN,Moon,test,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1971-10-06T00:00:00.213000Z,1971-10-07T00:00:03.080925Z,1.0,XA,S12,00,MHZ,81453e5c02bef85b63f73a116393f948


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
70,xa.s15.00.mhz.1974-12-15HR00_evid00169,NaN,NaN,evid00169,NaN,Moon,test,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1974-12-15T00:00:00.158000Z,1974-12-16T00:00:01.063660Z,1.0,XA,S15,00,MHZ,7445689798d974bd70098e2f4323e2f8
71,xa.s15.00.mhz.1974-12-15HR00_evid00170,NaN,NaN,evid00170,NaN,Moon,test,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1974-12-15T00:00:00.158000Z,1974-12-16T00:00:01.063660Z,1.0,XA,S15,00,MHZ,7445689798d974bd70098e2f4323e2f8


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
40,xa.s12.00.mhz.1972-12-02HR00_evid00083,1972-12-02T07:58:00.000000,28680.0,evid00083,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1972-12-02T00:00:00.439000Z,1972-12-03T00:00:00.137113Z,1.0,XA,S12,00,MHZ,f71fe89f14cb453f68e455de41638ab1
39,xa.s12.00.mhz.1972-12-02HR00_evid00341,NaN,NaN,evid00341,NaN,Moon,test,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1972-12-02T00:00:00.439000Z,1972-12-03T00:00:00.137113Z,1.0,XA,S12,00,MHZ,f71fe89f14cb453f68e455de41638ab1


,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type,body,subset,mseed_path,mseed_sampling_rate,mseed_starttime,mseed_endtime,mseed_calib,mseed_network,mseed_station,mseed_location,mseed_channel,mseed_hash
36,xa.s12.00.mhz.1972-07-17HR00_evid00067,1972-07-17T07:50:00.000000,28200.0,evid00067,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1972-07-17T00:00:00.184000Z,1972-07-17T23:59:59.882113Z,1.0,XA,S12,00,MHZ,dcc1ef5de869555ea2eb81f91ebe3b1e
37,xa.s12.00.mhz.1972-07-17HR00_evid00068,1972-07-17T21:56:00.000000,78960.0,evid00068,impact_mq,Moon,train,../data/raw/remote/space_apps_2024_seismic_det...,6.625,1972-07-17T00:00:00.184000Z,1972-07-17T23:59:59.882113Z,1.0,XA,S12,00,MHZ,dcc1ef5de869555ea2eb81f91ebe3b1e


## Extract Samples
1. we need to align sample rate amount the dataset, there are options (`#hyper-parameter`):
    - use the lowest available: 6.625 (Moon)
    - upsamples to the more common: 20Hz (Mars, Earth)
    - stay somewhere at the middle: 10Hz
2. Each trace will be split to the: 1) before event; 2) right after event; 3) long echo; 4) back to normal (`#hyper-parameter`)
    - 1,4 -- will be labeled as "noise"
    - 2 -- as "signal"
    - 3 -- will be through away (since we might have some mixed signal there) (duration is `#hyper-parameter`)

In [212]:
%%time

import datetime
from matplotlib import cm
import matplotlib.pyplot as plt

grouped = aggregated_df.groupby('mseed_path')

# `#hyper_parameter` which might be tuned (in seconds)
arrival_duration = 5000
signal_calm_down_seconds = 1.5 * arrival_duration

arrival_size = 64 * 1024 # 64k
minimal_size_of_interval = 8 * 1024 # 8K

target_sample_rate = 20 # upsample to align all records which we have

maxfreq = 4 # 4Hz -- during visualization notised that we rarely have useful signal above it (it also was used at NASA Demo Notebook)
minfreq = 0.05 #0.05Hz -- just used it from the NASA Demo Notebook

dataset_path = os.path.join(output_path, 'dataset')
label_field = 'time_abs(%Y-%m-%dT%H:%M:%S.%f)'

records = []

def mseed_load_and_scale(mseed_path):
    st = obspy.read(mseed_path)
    tr = st.traces[0].copy()
    starttime = tr.stats.starttime.datetime
    if tr.stats.sampling_rate != target_sample_rate:
        tr.interpolate(sampling_rate=target_sample_rate)
    tr.filter('bandpass',freqmin=minfreq, freqmax=maxfreq)
    # I guess it should be equal to `target_sample_rate` after interpolation above
    # sampling_rate = tr.stats.sampling_rate
    return tr.data, starttime

for mseed_path, group_data in tqdm(grouped):
    if len(group_data) == 1:
        row = group_data.iloc[0]
        # presumable there is only 1 event at the trace
        # so we can assume that the rest of data just noise
        if row['subset'] == 'test':
            # there is no labels so we do not know where signal or noise
            pass
        else:
            # the only clear case between signal and noise

            # st = obspy.read(mseed_path)
            # tr = st.traces[0].copy()
            # starttime = tr.stats.starttime.datetime
            # if tr.stats.sampling_rate != target_sample_rate:
            #     tr.interpolate(sampling_rate=target_sample_rate)
            # tr.filter('bandpass',freqmin=minfreq, freqmax=maxfreq)

            # # I guess it should be equal to `target_sample_rate` after interpolation above
            # sampling_rate = tr.stats.sampling_rate
            # tr_data = tr.data
            tr_data, starttime = mseed_load_and_scale(mseed_path)

            # TKTK: fix storage -- we cannot use mseed_starttime since it stored as datetime, but supposed to be utcdatetime
            # which isn't crtical for this application since we do not have such high precision here, but better to fix it anyway
            # starttime = row['mseed_starttime']
            arrival_time = datetime.datetime.strptime(row['time_abs(%Y-%m-%dT%H:%M:%S.%f)'],'%Y-%m-%dT%H:%M:%S.%f')
            arrival_starts_seconds = (arrival_time - starttime).total_seconds()
            arrival_starts_idx = int(arrival_starts_seconds * target_sample_rate)
            # arrival_ends_idx = int((arrival_starts_seconds + arrival_duration) * target_sample_rate)
            arrival_ends_idx = arrival_starts_idx + arrival_size
            arrival_ends_seconds = arrival_ends_idx / target_sample_rate

            # tr_times = tr.times()
            # signal_data = tr_data[arrival_starts_idx:arrival_ends_idx]
            # # print(signal_data.shape)
            # signal_times = tr_times[arrival_starts_idx:arrival_ends_idx]

            # fig, axes = plt.subplots(2, 1, figsize=(14, 2*4), sharex=True)
            # plot_spectrogram(
            #     signal_times,
            #     signal_data,
            #     target_sample_rate,
            #     axes[0],
            # )
            # axes[1].plot(tr_times, tr_data)
            # plt.show()

            markup = []

            # signal intervals

            markup.append(seismic.SeismicInterval(
                start_seconds=arrival_starts_seconds,
                start_idx=arrival_starts_idx,
                end_seconds=arrival_ends_seconds,
                end_idx=arrival_ends_idx,
                category=seismic.SIGNAL,
            ))

            # noise intervals
            ## before signal
            markup.append(seismic.SeismicInterval(
                start_seconds=0,
                start_idx=None,
                end_seconds=arrival_starts_seconds,
                end_idx=arrival_starts_idx,
                category=seismic.NOISE,
            ))

            ## after signal
            noise_starts_seconds = arrival_ends_seconds + signal_calm_down_seconds
            noise_starts_idx = int(noise_starts_seconds * target_sample_rate)
            noise_ends_seconds = row['mseed_endtime'] - row['mseed_starttime']
            noise_ends_idx = None
            
            markup.append(seismic.SeismicInterval(
                start_seconds=noise_starts_seconds,
                start_idx=noise_starts_idx,
                end_seconds=noise_ends_seconds,
                end_idx=noise_ends_idx,
                category=seismic.NOISE,
            ))

            record = seismic.SeismicRecord(
                record_id=row['filename'],
                mseed_path=mseed_path,
                space_body=row['body'],
                station=row['mseed_station'],
                markup=markup,
                waveform=tr_data,
            )

            seismic.store_record(dataset_path, record)
    else:
        # there are multiple events so we could have multiple signals at the trace
        # and it isn't clear where we supposed to have noise
        #
        # but we can still extract the signal
        tr_data, starttime = mseed_load_and_scale(mseed_path)

        markup = []

        for _, row in group_data.iterrows():
            if row.isna()[label_field]:
                continue

            # starttime = row['mseed_starttime']
            arrival_time = datetime.datetime.strptime(row[label_field],'%Y-%m-%dT%H:%M:%S.%f')
            arrival_starts_seconds = (arrival_time - starttime).total_seconds()
            arrival_starts_idx = int(arrival_starts_seconds * target_sample_rate)
            # arrival_ends_idx = int((arrival_starts_seconds + arrival_duration) * target_sample_rate)
            arrival_ends_idx = arrival_starts_idx + arrival_size
            arrival_ends_seconds = arrival_ends_idx / target_sample_rate

            markup.append(seismic.SeismicInterval(
                start_seconds=arrival_starts_seconds,
                start_idx=arrival_starts_idx,
                end_seconds=arrival_ends_seconds,
                end_idx=arrival_ends_idx,
                category=seismic.SIGNAL,
            ))

        # we only store records with known signals
        if len(markup) > 0:
            record = seismic.SeismicRecord(
                record_id=row['filename'],
                mseed_path=mseed_path,
                space_body=row['body'],
                station=row['mseed_station'],
                markup=markup,
                waveform=tr_data,
            )
            dump_filename = seismic.store_record(dataset_path, record)

  0%|          | 0/172 [00:00<?, ?it/s]

CPU times: user 3.4 s, sys: 770 ms, total: 4.17 s
Wall time: 5.85 s


## Output

In [18]:
import numpy as np
indexes = np.where(tr_times > arrival_seconds)
indexes, int(arrival_seconds * tr.stats.sampling_rate)

((array([486934, 486935, 486936, ..., 572412, 572413, 572414]),), 486933)

In [19]:
aggregated_df.to_csv(f'{output_path}/1_processing.csv')

# Appendix
Can remove before publish

In [16]:
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks, iirnotch, filtfilt, stft

def plot_spectrogram(tr_times_filt, tr_data_filt, sampling_rate, ax):
    # tr_times_filt = trace.times()
    # tr_data_filt = trace.data
    
    f, t, sxx = signal.spectrogram(tr_data_filt, sampling_rate)

    # Plot the spectrogram
    ## ignore outliers
    vmin = np.percentile(sxx, 1)
    vmax = np.percentile(sxx, 99.9)
    print('vmin:', vmin)
    print('vmax:', vmax)
    print(sxx.shape)
    vals = ax.pcolormesh(t, f, sxx, cmap=cm.jet, vmin=vmin, vmax=vmax)
    print(vals)
    # ax.set_xlim([min(tr_times_filt),max(tr_times_filt)])
    ax.set_xlabel(f'Time (Day Hour:Minute)', fontweight='bold')
    ax.set_ylabel('Frequency (Hz)', fontweight='bold')
    # ax.axvline(x=arrival, c='red')
    cbar = plt.colorbar(vals, orientation='horizontal')
    cbar.set_label('Power ((m/s)^2/sqrt(Hz))', fontweight='bold')

# END